In [3]:
import pandas as pd

# Load the CSV
df = pd.read_csv("cornbelters_xba_data.csv")
df = df[df['BatterTeam'] == 'Normal cornbelters']
# Define what counts as an AB (at-bat)
# Exclude: Walks, HitByPitch, Sacrifice, CatcherInterference, IntentionalWalk, etc.
# Include: Out, Error, FieldersChoice, Single, Double, Triple, HomeRun, etc.

# You may need to adjust these based on your dataset's PlayResult or KorBB values
not_ab = [
    "Walk", "IntentionalWalk", "HitByPitch", "SacrificeBunt", "SacrificeFly", "CatcherInterference"
]
# If you have a column for PlayResult or KorBB, use it to filter
ab_mask = ~df["PlayResult"].isin(not_ab)

# Group by batter and calculate xBA/AB
result = (
    df[ab_mask]
    .groupby("Batter")
    .agg(
        AB=("PlayResult", "count"),
        xBA_sum=("xBA", "sum")
    )
    .assign(xBA_per_AB=lambda x: x["xBA_sum"] / x["AB"])
    .reset_index()
    .sort_values("xBA_per_AB", ascending=False)
)

result["xBA_percentile"] = result["xBA_per_AB"].rank(pct=True)

# Sort by xBA_per_AB descending
result = result.sort_values("xBA_per_AB", ascending=False)

# Show the result
print(result[["Batter", "xBA_per_AB", "xBA_percentile", "AB"]])

             Batter  xBA_per_AB  xBA_percentile  AB
11   Tyler Thompson    0.382940        1.000000  29
0       Caleb Royer    0.373434        0.923077  21
2      Cole Freeman    0.364211        0.846154  25
4     Jackson Smith    0.305832        0.769231  37
5    Jackson Stanek    0.295739        0.692308  21
3        Jack Novak    0.275862        0.615385  29
10     Shea Zbrozek    0.261134        0.538462  26
12   Zach Zychowski    0.259109        0.461538  26
8   Nolan Mccrossin    0.241228        0.384615  24
7     Michael Kuska    0.205742        0.307692  11
1    Charlie Graham    0.157895        0.230769  11
9      Ryan Colucci    0.056680        0.153846  13
6     Kannan Kleine    0.026316        0.076923   6


In [4]:
result.to_csv("cornbelters_xba.csv" , index=False)

In [1]:
import pandas as pd
df = pd.read_csv('cornbelters_xba_data.csv')
df = df[['PitchUUID','xBA']]
df.to_csv('../BaseballSavant/cornbelters_xba.csv',index = False)